Elastic Search Script


In [30]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install pillow numpy elasticsearch tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install elasticsearch
!pip install requests beautifulsoup4 tqdm pillow
!pip install requests lxml tqdm



Looking in indexes: https://download.pytorch.org/whl/cpu
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-r64nxgg7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-r64nxgg7
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [35]:
!pip install sentence-transformers


In [36]:
import clip
import torch
from PIL import Image
import numpy as np
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import numpy as np


client = Elasticsearch(
    "https://33cb0c87675645148676794cbec9680d.us-east-1.aws.found.io:443",
    api_key="a3FSbU5Kb0JJWUFJRE8wZWRYRmg6VDRSYjhROXVPU1JBSVhoemxsNHpZQQ=="
)

index_name = "multimodal_media"
print(client.info().body)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL, PREPROCESS = clip.load("ViT-L/14", device=DEVICE)  # 768-dim



# Load E5 model for long text
text_model = SentenceTransformer("intfloat/e5-base-v2")

def embed_text(text: str) -> np.ndarray:
    """
    Generate an embedding for long webpage text using E5 model.
    """
    # Clean + prefix (E5 expects a prompt style)
    text = text.strip()
    text = "passage: " + text
    emb = text_model.encode(text, normalize_embeddings=True)
    return np.array(emb)

# def embed_text(text: str) -> np.ndarray:
#     with torch.no_grad():
#         tokens = clip.tokenize([text]).to(DEVICE)
#         feats = MODEL.encode_text(tokens)
#         feats = feats / feats.norm(dim=-1, keepdim=True)
#     return feats[0].cpu().numpy().astype(np.float32)

def embed_image(path_or_pil) -> np.ndarray:
    img = path_or_pil if isinstance(path_or_pil, Image.Image) else Image.open(path_or_pil).convert("RGB")
    img = PREPROCESS(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        feats = MODEL.encode_image(img)
        feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats[0].cpu().numpy().astype(np.float32)

def index_image(doc_id: str, title: str, url: str, vector):
    body = {
        "id": doc_id,
        "media_type": "image",
        "title": title,
        "url": url,
        "vector_clip": vector.tolist(),
        "metadata": {}
    }
    client.index(index=index_name, id=doc_id, document=body)
    print(f"Indexed image with id = {doc_id}")


# TEMPORARY TEST
if __name__ == "__main__":
    print(len(embed_text("hello world")))  # should print 768


{'name': 'instance-0000000004', 'cluster_name': '33cb0c87675645148676794cbec9680d', 'cluster_uuid': 'Eq0DnWFDQxmO_d_YkDYgLQ', 'version': {'number': '9.0.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '112859b85d50de2a7e63f73c8fc70b99eea24291', 'build_date': '2025-04-08T15:13:46.049795831Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

768


In [37]:
from PIL import Image
import requests
from io import BytesIO

# Example image URL
image_url = "https://member.acg.aaa.com/content/dam/acg/images/home/pr-home/hero-splash-3.jpg"  #  replace with your image URL

# Download and open image from URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# Generate vector
vec = embed_image(img)
print("Image embedding length:", len(vec))

# Index into Elasticsearch
index_image(
    doc_id="img_001",
    title="Test Image",
    url=image_url,
    vector=vec
)


Image embedding length: 768
Indexed image with id = img_001


In [38]:
def search_by_text(query_text: str, size: int = 5):
    # Convert text → vector
    qvec = embed_text(query_text).tolist()

    # Build kNN search query
    body = {
        "size": size,
        "knn": {
            "field": "vector_clip",
            "query_vector": qvec,
            "k": max(50, size),
            "num_candidates": 500
        },
        "_source": ["id", "media_type", "title", "url", "metadata"]
    }

    # Execute search
    res = client.search(index=index_name, body=body)

    # Simplify the hits
    results = [
        {
            "id": hit["_source"].get("id"),
            "score": hit["_score"],
            "title": hit["_source"].get("title"),
            "url": hit["_source"].get("url")
        }
        for hit in res["hits"]["hits"]
    ]
    return results


In [39]:
if __name__ == "__main__":
    # Example: text search instead of manual upload
    query = input("Enter your search text: ")
    results = search_by_text(query)

    print("\n Search Results:")
    for i, r in enumerate(results, 1):
        print(f"{i}. [{r['score']:.4f}] {r['title']} → {r['url']}")


Enter your search text: insu

 Search Results:
1. [0.5323] AAA roadside assistance technician replacing a flat tire. → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/technician-fixes-flat?ts=1748959195349&dpr=off
2. [0.5304] A man and a woman purchasing something online with a credit card → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/UAC-17048_image3_1920x2560_less?ts=1754936470091&bfc=on&network=on&dirt_feature=smart_imaging&dpr=off
3. [0.5279] A woman smiling as she sits in the drivers seat of her car putting her seatbelt on.  → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/auto-loans-1?ts=1759868502039&bfc=on&network=on&dirt_feature=smart_imaging&dpr=off
4. [0.5182] Roadside Assistance → https://www.dev1.acg.aaa.com/content/dam/oneacg/global/roadside-assistance/roadside-desktop-v1.png
5. [0.5176] Test Image → https://member.acg.aaa.com/content/dam/acg/images/home/pr-home/hero-splash-3.jpg


In [42]:


from xml.etree import ElementTree

def crawl_from_sitemap(sitemap_url: str, limit: int = 500):
    """
    Reads the sitemap XML, extracts all <loc> URLs,
    fetches each page, extracts text/images/videos,
    and indexes them into Elasticsearch.
    """
    print(f" Reading sitemap: {sitemap_url}")
    resp = requests.get(sitemap_url, timeout=10)
    xml_root = ElementTree.fromstring(resp.content)

    urls = [elem.text for elem in xml_root.iter("{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
    print(f" Found {len(urls)} URLs in sitemap")

    count = 0
    for url in tqdm(urls[:limit], desc="Indexing pages"):
        try:
            r = requests.get(url, timeout=8)
            if "text/html" not in r.headers.get("Content-Type", ""):
                continue

            soup = BeautifulSoup(r.text, "html.parser")
            page_title = soup.title.string if soup.title else url
            page_text = " ".join(t.get_text(strip=True) for t in soup.find_all(["p", "h1", "h2", "h3"]))

            if len(page_text) > 100:
                vec = embed_text(page_text)
                client.index(index=index_name, id=f"page_{count}", document={
                    "id": f"page_{count}",
                    "media_type": "page",
                    "title": page_title,
                    "url": url,
                    "vector_clip": vec.tolist(),
                    "metadata": {"length": len(page_text)}
                })
                count += 1

            # Optional: also grab images
            for img_tag in soup.find_all("img", src=True):
                img_url = urljoin(url, img_tag["src"])
                try:
                    img_resp = requests.get(img_url, timeout=5)
                    img = Image.open(BytesIO(img_resp.content)).convert("RGB")
                    vec = embed_image(img)
                    index_image(
                        doc_id=f"img_{count}",
                        title=img_tag.get("alt", f"Image {count}"),
                        url=img_url,
                        vector=vec
                    )
                    count += 1
                except Exception:
                    pass

        except Exception as e:
            print(f" Skipping {url}: {e}")

    print(f"\n Finished indexing {count} items from sitemap.")


In [34]:
if __name__ == "__main__":
    sitemap_url = input("Enter the sitemap url ")
    crawl_from_sitemap(sitemap_url, limit=500)


Enter the sitemap url https://dev1.meemic.com/sitemap.xml
 Reading sitemap: https://dev1.meemic.com/sitemap.xml
 Found 208 URLs in sitemap


Indexing pages:   0%|          | 1/208 [00:00<02:18,  1.50it/s]

 Skipping https://dev1.meemic.com: Input Resources Resources Resources Resources Resources Resources Resources Resources Resources Resources Helping Safeguard Your Future: Exclusive Insurance Benefits for Teachers At Meemic, we’re committed to providing teachers and educators with quality insurance solutions tailored to their needs. With a focus on service and value, we offer the protection and support you deserve throughout your career. We are currently only offering Meemic Insurance to residents in Michigan. Meemic is growing and we hope to provide protection for educators in more states in the near future. Until then, please visit our partners at AAA to receive a quote. Meemic Products & Services Auto Coverage for Educators & Staff Meemic specializes in providing comprehensive auto insurance tailored specifically for educators and staff, ensuring peace of mind on the road. Learn More Help Protect Your Property & Loved Ones Learn about the simple steps you can take to improve safety 

Indexing pages:   1%|          | 2/208 [00:01<02:15,  1.52it/s]

 Skipping https://dev1.meemic.com/auto-insurance: Input Resources Resources Resources Resources Resources Resources Resources Resources Resources Resources Car Insurance Built for Educators Founded nearly 75 years ago by teachers and for teachers, Meemic is proud to support today’s teachers with high-quality, dependable auto insurance. See our special discounts and benefits for teachers with an online auto insurance quote or by connecting with an agent. We are currently only offering Meemic Insurance to residents in Michigan. Meemic is growing and we hope to provide protection for educators in more states in the near future. Until then, please visit our partners at AAA to receive a quote. Who is Eligible to Join Meemic? Teachers, Educators & Staff Meemic Membership extends beyond teachers to include all educational employees, such as: Retired Teachers At Meemic, we truly believe there is no calling more noble or more important than that of the educational community. Whether you’ve been

...

 Skipping https://dev1.meemic.com/back-to-school/back-to-school-terms: Input Resources Resources Resources Resources Resources Resources Resources Resources Resources Resources Meemic’s $10,000 Back to School Sweepstakes (“Promotion”)Meemic Insurance Company, 1685 N. Opdyke Rd., Auburn Hills, MI 48326 (“Sponsor”)OFFICIAL RULES & REGULATIONSNO QUOTE OR PURCHASE NECESSARY TO ENTER OR WIN. A PURCHASE WILLNOT IMPROVE AN INDIVIDUAL’S CHANCES OF WINNING. VOID WHEREPROHIBITED BY LAW.  ELIGIBILITY: The Promotion is only open to legal residents in the state of Michigan who, at the time of entry, have reached the age of majority in their state of residence and are educators or school employees currently employed at one of the following: You do not need to be a Member of Meemic Insurance Company to enter. Educators and school employees of home schools are not eligible to enter. A home school is defined as a building that also serves as a private residence and/or serves fewer than 50 students. Emp

Indexing pages: 100%|█████████▉| 207/208 [01:47<00:00,  1.83it/s]

 Skipping https://dev1.meemic.com/jotform-test: Input Resources Resources Resources Resources Resources Resources Resources Resources Resources Resources Meemic Insurance Company provides insurance for teachers and educators in Michigan. Discounts and savings opportunities subject to eligibility requirements. Coverage is subject to all policy terms, conditions, exclusions and limitations. Subject to underwriting eligibility requirements. Home insurance, auto insurance, and recreational vehicle insurance underwritten by Meemic Insurance Company, 1685 N. Opdyke Rd., Auburn Hills, MI 48326. Umbrella insurance underwritten by MemberSelect Insurance Company, 1 Auto Club Drive, Dearborn, MI 48126. Motorcycle insurance and boat insurance are available through ACG South Insurance Agency, LLC and underwritten by non-affiliated insurance companies.     is too long for context length 77


Indexing pages: 100%|██████████| 208/208 [01:47<00:00,  1.93it/s]

 Skipping https://dev1.meemic.com/home-auto-bundle: Input Resources Resources Resources Resources Resources Resources Resources Resources Resources Resources Bundle Your Car & Home Insurance Today! At Meemic, we believe Michigan educators deserve more than just words of thanks, they deserve meaningful savings. That’s why we offer discounts when you bundle your auto and home insurance. It’s our way of showing appreciation through real value. Connect with your local Meemic agent or start your quote online today. Who is Eligible to Join Meemic? Teachers, Educators & Staff Meemic Membership extends beyond teachers to include all educational employees, such as: Retired Teachers At Meemic, we truly believe there is no calling more noble or more important than that of the educational community. Whether you’ve been teaching for just a few years or an entire lifetime, being an educator is in your blood. That’s something that doesn’t go away when you retire — and neither does your opportunity to